In [284]:
# Load data
import re
from itertools import chain # this consumes iterable

with open("../data/day16.txt", "r") as f:
    input = f.read()

def parse(input):
    input_list = input.split("\n\n")
    rules = input_list[0].split("\n")
    my_ticket = [int(value) for value in input_list[1].split("\n")[1].split(",")]
    nearby_tickets = [[int(value) for value in ticket.split(",")] for ticket in input_list[2].split("\n")[1:]]

    regex = re.compile(r"(.*): (\d+)-(\d+) or (\d+)-(\d+)") # named groups to dict?
    rule_dict = {}
    for rule in rules:
        data = regex.search(rule).groups()
        rule_name = data[0]
        allowed = list(range(int(data[1]), int(data[2]) + 1)) + list(range(int(data[3]), int(data[4]) + 1))
        rule_dict[rule_name] = allowed

    full_rule_set = set().union(*rule_dict.values())
    return rule_dict, nearby_tickets, my_ticket

rule_dict, nearby_tickets, my_ticket = parse(input)

#zone: 34-521 or 534-971
assert 33 not in rule_dict["zone"] 
assert 34 in rule_dict["zone"]
assert 521 in rule_dict["zone"]
assert 522 not in rule_dict["zone"]
assert 533 not in rule_dict["zone"]
assert 534 in rule_dict["zone"]
assert 971 in rule_dict["zone"]
assert 972 not in rule_dict["zone"]

In [285]:
# Puzzle 1 (expanded/changed to retain valid tickets)
import numpy as np

errors = []
valid_tickets = []
for ticket in nearby_tickets:
    ticket_errors = [value for value in ticket if value not in full_rule_set]
    if ticket_errors:
        errors.append(*ticket_errors)
    else:
        valid_tickets.append(ticket)

print(sum(errors))
print(len(valid_tickets))

21081
190


In [286]:
# Puzzle 2
import numpy as np

# Get ticket array
ticket_array = np.array(valid_tickets)
print(ticket_array)

# Get all possible rules for every field
rule_options = []
for n in range(0, np.shape(ticket_array)[1]):
    rule_option = [rule_name for rule_name, values in rule_dict.items() if set(ticket_array[:,n]).issubset(set(values))]
    rule_options.append(rule_option)

# Eliminate rules (can probably be done smarter)
final_rules = {}
while [ro for ro in rule_options if ro]:
    k = 0
    for rules in rule_options:
        if len(rules) == 1:
            to_remove = rules[0]
            final_rules[k] = to_remove
            for rules_deep in rule_options:
                if to_remove in rules_deep:
                    rules_deep.remove(to_remove)
        k += 1

print(final_rules)

# Filter keys on departure
np.prod([my_ticket[key] for key, field in final_rules.items() if field.startswith("departure")])

[[279 705 188 ... 501 808 245]
 [ 51 436 407 ... 552 164 813]
 [407 242 909 ... 198 806  64]
 ...
 [348 556 567 ... 631 560 344]
 [515 391 442 ... 492 494 485]
 [409 442 390 ... 684 632 192]]
{0: 'route', 12: 'seat', 4: 'arrival station', 5: 'duration', 1: 'arrival track', 8: 'train', 15: 'class', 19: 'departure track', 13: 'departure time', 7: 'departure platform', 16: 'departure date', 14: 'departure station', 2: 'departure location', 17: 'price', 11: 'wagon', 18: 'zone', 9: 'row', 6: 'arrival platform', 3: 'type', 10: 'arrival location'}


314360510573